In [ ]:
## DTS Complete Streets
## 
## 1 - Calculate Unconstrained Modal Widths
##
## rmangan

**DTS - Complete Streets**
## XX - Modal Widths / Complete Streets Typology Assignment



**Purpose**

1. Develop individual modal widths per modal type, then summarize modal widhts into an "unconstrained" modal width metric. Various other metrics are then calculated using width esitmates and linear line length. (modal area, unconstrained modal area, exceedence area etc.)

2. Assign Complete Street "Typologies" using various attributes from the previously compiled "modal_composite" dataset.



**Global Assumptions and Notes**

global assumptions and notes go here detailed assumptions should go inline with specific code segments.
1. Item
2. item
3. item
4. item

**Non-Standard Python Modules utilized**
1. ArcPy (ArcGIS Pro) required
2. Pandas (optional - only used for QC purposess and data exploration)


**Author:** rmangan


In [ ]:
# import modules
import arcpy
import os
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.gis import GIS

In [ ]:
# set environment setttings
arcpy.env.workspace = "Z:\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\modal\Modal_Composite4.gdb"
arcpy.env.overwriteOutput = True

In [ ]:
# define variables
input_gdb_path = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\Input_Data.gdb"

scratch_gdb_path = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\scratch_GDBs\modal_composite_scratch.gdb"

output_gdb_path = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\scratch_GDBs\modal_composite_output.gdb"


# Input Datasets
#Ped_Plan = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\Input_Data.gdb\OPP_ModalPriority_081720"

Ped_Plan = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\scratch_GDBs\OPP_domain_processing.gdb\OPP_ModalPriority_081720_output"

Ped_Improve = os.path.join(input_gdb_path,"OPP_CandidateUpgrades_090420Dissolve_101320")

Ped_Add = os.path.join(input_gdb_path,"OPP_CandidateWalkways_091520Dissolve_update101620")

Bike_Exist = os.path.join(input_gdb_path,"BIKEPLAN_Existing_Bikeways")

Bike_Proposed = os.path.join(input_gdb_path,"BIKEPLAN_Proposed_Bikeways")

Bike_ReDev = os.path.join(input_gdb_path,"BIKEPLAN_Redevelopment_Bikeways")

RCL = os.path.join(input_gdb_path,"RCL_Public_Street_Centerline_20201013")

TransCAD_2020 = os.path.join(input_gdb_path,"MPO_TRANSCAD_network2020")

TransCAD_2020_aloha_stadium = os.path.join(input_gdb_path,"MPO_TRANSCAD_network2020_aloha_stadium")

TransCAD_2040 = os.path.join(input_gdb_path,"MPO_TRANSCAD_network_adjusted2040")

modal_composite_02 = os.path.join(scratch_gdb_path,"modal_composite_02")

HPMS = os.path.join(input_gdb_path,"HPMS_RCL_join")

HERE = os.path.join(input_gdb_path,"RCLs_with_NAVOIDs_35buff_10clip")

medians = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\scratch_GDBs\median_classification.gdb\modal_comp_05_median_classification"

modal_composite = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\Modal\Modal Composite 5_3.gdb\modal_composite_05_3"

modal_composite_url = r"https://services6.arcgis.com/2cZSk3EXXiOHcbOl/ArcGIS/rest/services/Modal_Composite_5_1_publish_gdb/FeatureServer/0"

modal_composite_url_copy = r"https://services6.arcgis.com/2cZSk3EXXiOHcbOl/ArcGIS/rest/services/Modal_Composite_5_1_publish_gdb/FeatureServer/0"

zoning = r"\\dc1vs01\GISProj\B\Base\HI\Honolulu_City_County\zoning\CCH_zoning.gdb\zoning_LUO_20210505"

In [ ]:
#featureset and updte testing against AGOL FS

#generate token
token = gis._con.token

print("Token: {}".format(str(token)))

modal_composite_fs = modal_composite_url_copy + "?token={}".format(token)

print(modal_composite_fs)

featureset = arcpy.FeatureSet()

fs = gis.content.get('6655329b1fb34a91b419b8d002588524')
fs.url

# lyr = arcpy.management.MakeFeatureLayer(modal_composite, "modal_comp_lyr")[0]


featureset.load(modal_composite_fs)

In [ ]:
type(modal_composite_fs)

In [ ]:
fields = ["SEGMENTID","FULLNAME"]

count = 0

with arcpy.da.SearchCursor(modal_composite_fs, fields) as cursor:
    for row in cursor:
        print(row[1])
        count += 1

        
print(count)
        

In [ ]:
#create num fields for modal width calcualtions

#copy input dataset while testing functions
print("copying modal input for processing...")
modal_processing_temp = arcpy.CopyFeatures_management(modal_composite_04,"modal_composite_04a")
print("done")

#add field for assumed lanes for streets that do not have any lane data from TransCAD
arcpy.AddField_management(modal_composite_05a,field_name="lanes_assumed",field_type="SHORT",field_alias = "Lanes (assumed)")

#store field, field alias in dict
new_fields = {"modal_width_ped_exist":  "Modal Width - Ped (Existing)",
              "modal_width_ped_prop":   "Modal Width - Ped (Proposed)",
              "modal_width_bike_exist": "Modal Width - Bike (Existing)",
              "modal_width_bike_prop":  "Modal Width - Bike (Proposed)",
              "modal_width_auto_exist": "Modal Width - Auto (Existing)",
              "modal_width_auto_prop":  "Modal Width - Auto (Proposed)",
              "modal_width_bus_exist":  "Modal Width - Bus (Existing)",
              "modal_width_bus_prop":   "Modal Width - Bus (Proposed)",
              "modal_width_park_exist": "Modal Width - Parking (Existing)",
              "modal_width_park_prop":  "Modal Width - Parking (Proposed)",
              "modal_width_medians":    "Modal Width - Medians",
              "modal_width_max":        "Modal Width - Total Unconstrained"}

#loop through dict and add fields
for key,value in new_fields.items():
    print("Adding field {0}".format(key))
    arcpy.AddField_management(modal_composite_05a,field_name=key,field_type="SHORT", field_alias = value)
    

          
print("Field additions complete")


In [ ]:
#add field auto widths QC

#store field, field alias in dict
new_fields = {
              "modal_width_auto_rule": "Modal Width - Auto (Logic)"
}

#loop through dict and add fields
for key,value in new_fields.items():
    print("Adding field {0}".format(key))
    arcpy.AddField_management(modal_composite,field_name=key,field_type="TEXT", field_alias = value)
    

          
print("Field additions complete")

In [ ]:
#fix bike proposed missing fac_type field - MOVED TO SCRIPT 04

bike_prop_intr = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\scratch_GDBs\modal_composite_scratch.gdb\Bike_Proposed_RCL_intersect"

bike_prop_alias = {
    "Fac_Type":"Facility Type (Bike Proposed)"
}

#rename bike proposed field
for key, value in bike_prop_alias.items():
    new_field_name = str(key)+"_BP"
    print("assigning alias {} to field {}, field renamed to {}".format(value,key,new_field_name))
    arcpy.AlterField_management(bike_prop_intr,key,new_field_name, value)
    
    
#join Bike Proposed intersect results
print("Joining Bike Proposed fields...")
join_target = modal_composite_05a
join_target_field = "SEGMENTID"
join_table = bike_prop_intr
join_table_field = "SEGMENTID"
join_fields = ["Fac_Type_BP"]

arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)
print("Join Fields complete.")
    
print("Done")



In [ ]:
#join fields from median classification
print("Joining Median fields...")
join_target = modal_composite_05a
join_target_field = "SEGMENTID"
join_table = medians
join_table_field = "SEGMENTID"
join_fields = ["median_type","median_width","progression"]

arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)
print("Join Fields complete.")
    
print("Done")


In [ ]:
#Join fieds from HMPS
print("Joining HPMS fields...")
join_target = modal_composite_05_2
join_target_field = "SEGMENTID"
join_table = HPMS
join_table_field = "SEGMENTID"
join_fields = ["aadt","route_id","funsystem", "funsysdesc","factype","factypedes","thrulanes"]

arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)
print("Join HPMS Fields complete.")

#Join fields from HERE
print("Joining HERE fields...")
join_target = modal_composite_05_2
join_target_field = "SEGMENTID"
join_table = HERE
join_table_field = "SEGMENTID"
join_fields = ["SPEED","SPD_LIMIT","MIN_SPD_LIMIT", "MAX_SPD_LIMIT"]

arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)
print("Join HERE Fields complete.")
    
print("Done")


In [ ]:
#update field alias for HPMS and HERE fields

update_alias = {
    "aadt":"HPMS - Annual Average Daily Traffic, Two Directions",
    "route_id":"HPMS - Route Number",
    "funsystem":"HPMS - Functional System Code",
    "funsysdesc":"HPMS - Functional System Code Description",
    "factype":"HPMS - Facility Type Code",
    "factypedes":"HPMS - Facility Type Code Description",
    "thrulanes":"HPMS - Number of Through Lanes, Two Directions",
    "SPD_LIMIT":"Speed Limit (HERE)",
    "MIN_SPD_LIMIT":"Speed Limit (HERE - low)",
    "MAX_SPD_LIMIT":"Speed Limit (HERE - high)"
}

#rename bike proposed field
for key, value in update_alias.items():
    new_field_name = str(key)+"_BP"
    print("assigning alias {0} to field {1}".format(value,key))
    arcpy.AlterField_management(modal_composite_05_2,key,new_field_alias=value)
    
print("alias assignment done")

### Calculate Modal Widths for Auto Traffic

- Calculate existing modal with for auto traffic using lane count x average lane width into `modal_width_auto_exist`. 

- If VOC Max from TransCAD 2040 > 1.0, assume two-way streets have two additonal lanes added (22' additional width), and one-way streets have one additional lane added (11' additonal width). Proposed width is calcualted into `modal_width_auto_prop`

- If VOC Max from TransCAD 2040 is <1.0, existing width is calculted into proposed width


**Assumptions, Notes, and Parameters**
1. Average lane width parameter =  11' wide.
2. Street segments with no lane data have the following additional assumptions applied, in order of implementation: 
    1. If `ONEWAY = 0` (street is two-way) and `mean_row < 100` assume 2 lanes total for line segment. Adjust with `param_two_way`
    2. If `ONEWAY = 0` (street is two-way) and `mean_row > 100` disregard (segments are typically edge cases)
    3. If `ONEWAY IN (1,2)` (street is one-way) and `mean_row < 75` assume 1 lane total for line segment. Adjust with `param_one_way`
    4. If `ONEWAY IN (1,2)` (street is one-way) and `mean_row > 75` disregard (segments are typically edge cases)
3. All street segments that have lane data, but do not have TransCAD VOC Max data have `modal_width_auto_exist` calculated into `modal_width_auto_prop`
4. Remaining street segments with no lane data, no mean_row calcualtion, and no TransCAD data, but have `CLASS = A41` are assumed to be 2 lanes, 22' width (regardless of oneway/two-way)
4. All street segments that were unable to have mean_row calculated, and do not have lane data are ignored


In [ ]:
#Conditional Re-Write
#PARAMETERS
param_one_way = 75.0
param_two_way = 100.0

#fields to expose to update cursor
VOC_fields = ["AB_VOC_EA_1", #0
              "BA_VOC_EA_1", #1
              "AB_VOC_AM_1", #2
              "BA_VOC_AM_1", #3
              "AB_VOC_MD_1", #4
              "AB_VOC_MD_1", #5
              "AB_VOC_PM_1", #6
              "BA_VOC_PM_1", #7
              "AB_VOC_EV_1", #8
              "BA_VOC_EV_1", #9
              "ONEWAY",      #10
              "modal_width_auto_exist", #11
              "modal_width_auto_prop",  #12
              "AB_LANE",     #13
              "BA_LANE",     #14
              "SEGMENTID",   #15
              "ln_exist_num",#16
              "AB_VOC_MAX",  #17
              "BA_VOC_MAX",  #18 
              "mean_row_2020", #19
              "lanes_assumed", #20,
              "CLASS",         #21
              "VOC_MAX",    #22
              "thrulanes",   #23  HPMS Lanes
              "modal_width_auto_rule" #24
             ]

print("Update rows for Traffic widths...")
count = 0
count_01_01 = 0
count_01_02 = 0
count_01_03 = 0
count_01_04 = 0
count_02_01 = 0
count_02_02 = 0
count_02_03 = 0
count_02_04 = 0
count_03_00 = 0
count_03_01 = 0
count_03_02 = 0
count_03_03 = 0
count_03_04 = 0
count_03_04_01 = 0
count_03_05 = 0
voc_above = 0
voc_below = 0
voc_null = 0
err_count = 0


#where_clause = '("VOC_max" is NOT null) AND ("VOC_Max" >= 1.0)'


with arcpy.da.UpdateCursor(modal_composite, VOC_fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            
            #CONDITIONAL GROUP 01
            
            #VOC > 1.0
            #TransCAD Lanes
            #One Way Streets
                   
            if(
                (row[22] is not None) and
                (row[22] >= 1.0) and
                (row[13] is not None) and
                (row[14] is not None) and    #TransCAD Lanes
                (row[10] in [1,2])
            ):

                #take max of the two lane values and multiply by 11
                current_width = ((max(row[13],row[14]))*11) + 2
                expanded_width = current_width + 11
            
                #print("One Way - VOC Max:{0}, Current:{1}, Proposed: {2}".format(VOC_max,current_width,expanded_width))
                
                cond = "01_01" 
                row[11] = current_width
                row[12] = expanded_width
                row[24] = cond
                voc_above +=1 
                count_01_01 +=1
                    
                 
            
            #VOC > 1.0
            #One Way Streets
            #ASSUMED Lanes
            elif (
                (row[22] is not None) and
                (row[22] >= 1.0) and        #VOC
                (row[13] is None) and
                (row[14] is None) and             #TransCAD Lanes IS NULL
                (row[10] in [1,2]) and                              #oneway indicator
                (row[19] is not None) and
                (row[19] < param_one_way)
            ):
                
                current_width = 11
                expanded_width = 22
                #print("One Way - VOC Max:{0}, Current:{1}, Proposed: {2} Assumed 1 lane".format(VOC_max,current_width,expanded_width))
                
                cond = "01_02"
                
                row[11] = current_width
                row[12] = expanded_width
                row[20] = 1
                row[24] = cond
                
                voc_above +=1 
                count_01_02 +=1
                
                
                
            #VOC > 1.0
            #Two Way Streets
            #TransCAD Lanes
            elif (
                (row[22] is not None) and
                (row[22] >= 1.0) and    #VOC
                (row[13] is not None) and
                (row[14] is not None) and  #TransCAD Lanes
                (row[10] == 0)                                  #two-way indicator
            ):
                
                current_width = ((row[13] + row[14])*11) + 2
                expanded_width = current_width + 11
                #print("Two Way - VOC Max:{0}, Current:{1}, Proposed: {2}".format(VOC_max,current_width,expanded_width))
                
                cond = "01_03"
                row[11] = current_width
                row[12] = expanded_width
                row[24] = cond
                voc_above +=1
                count_01_03 +=1
                
                
            #VOC > 1.0
            #Two Way Streets
            #ASSUMED 2 Lanes
            elif (
                (row[22] is not None) and
                (row[22] >= 1.0) and       #VOC
                (row[13] is None) and
                (row[14] is None) and             #TransCAD Lanes IS NULL
                (row[10] == 0) and                                  #two-way indicator
                (row[19] is not None) and
                (row[19] < param_two_way)  #ROW width parameter
            ):
                
                current_width = 24
                expanded_width = 44
                #print("One Way - VOC Max:{0}, Current:{1}, Proposed: {2} Assumed 2 lanes".format(VOC_max,current_width,expanded_width))
                
                cond = "01_04"
                row[11] = current_width
                row[12] = expanded_width
                row[20] = 2
                row[24] = cond
                voc_above +=1
                count_01_04 +=1
                
                
            
            #CONDITIONAL GROUP 02
            
            #VOC < 1.0
            #One Way Streets
            #TransCAD Lanes
            elif (
                (row[22] is not None) and     #VOC
                (row[13] is not None) and  #AB_Lane
                (row[14] is not None) and  #BA_Lane
                (row[22] < 1.0) and         #VOC
                (row[10] in [1,2])
            ):
                
                current_width = ((max(row[13],row[14]))*11) + 2  
                
                #print("One Way - VOC Max <1.0, Current:{0}, Proposed: {1} - {2}-{3}".format(current_width,expanded_width, row[17], row[18]))
                
                cond = "02_01"
                row[11] = current_width
                row[12] = current_width
                row[24] = cond
                count_02_01 +=1
                voc_below += 1
                
                
            
            #VOC < 1.0
            #One Way Streets
            #ASSUMED One Lane

            elif (
                (row[22] is not None) and
                (row[22] < 1.0) and          #VOC
                (row[13] is None) and
                (row[14] is None) and               #TransCAD Lanes IS NULL
                (row[10] in [1,2]) and                                      #one-way indicator
                (row[19] is not None) and
                (row[19] < param_one_way)#one-way width assumption parameter 
            ): 
                
                current_width = 13
                
                #print("One Way - VOC Max:{0}, Current:{1}, Proposed: {2} Assumed 1 lane".format(VOC_max,current_width,expanded_width))
                
                cond = "02_02"
                row[11] = current_width
                row[12] = current_width
                row[20] = 1
                row[24] = cond
                count_02_02 +=1
                voc_below += 1
                
                
            #VOC < 1.0
            #Two Way Streets
            #TransCAD Lanes
            elif (
                (row[22] is not None) and   #VOC
                (row[13] is not None) and
                (row[14] is not None) and   #TransCAD Lanes
                (row[22] < 1.0) and       
                (row[10] == 0)
            ):
                
                current_width = ((row[13] + row[14])*11)+2
                
                #print("Two Way - VOC Max <1.0, Current:{0}, Proposed: {1}".format(current_width,expanded_width))
                
                cond = "02_03"
                row[11] = current_width
                row[12] = current_width
                row[24] = cond
                count_02_03 +=1
                voc_below += 1
                
                
            #VOC < 1.0
            #Two Way Streets
            #ASSUMED Two Lanes
            elif (
                (row[22] is not None) and
                (row[22] < 1.0) and       #VOC
                (row[13] is None) and
                (row[14] is None) and            #TransCAD Lanes
                (row[10] == 0) and                                 #two-way indicator
                (row[19] is not None) and
                (row[19] < param_two_way) #ROW parameter
            ):
                
                current_width = 24
                
                #print("One Way - VOC Max:{0}, Current:{1}, Proposed: {2} Assumed 2 lanes".format(VOC_max,current_width,expanded_width))
                
                cond = "02_04"
                row[11] = current_width
                row[12] = current_width
                row[20] = 2
                row[24] = cond
                count_02_04 +=1
                voc_below += 1
                
                    
            #CONDITIONAL GROUP 03
            
            
            #VOC Null
            #One Way Streets
            #HPMS Lanes
            elif (
                (row[22] is None) and        #VOC is NULL
                (row[23] is not None) and  #HPMS lanes are not null
                (row[10] in [1,2] )
            ):
 
                current_width = (row[23]*11)+2
                
                #print("One Way - VOC Max:{0}, Current:{1}, Proposed: {2} Assumed 1 lane".format(VOC_max,current_width,expanded_width))
                cond = "03_00"
                row[11] = current_width
                row[12] = current_width
                row[20] = 1
                row[24] = cond
                
                voc_above +=1 
                count_03_00 +=1
                
            
            #VOC Null
            #One Way Street
            #TransCAD Lanes
            #note this most likely does not occur but was implemented to catch records if they do occur
            elif (
                (row[22] is not None) and
                (row[13] is not None) and 
                (row[14] is not None) and
                (row[10] in [1,2])
            ):
        
                current_width = (max(row[13],row[14])) * 11
                #print("One Way - VOC Max: NULL, Current:{0}, Proposed: N/A".format(current_width))
                
                cond = "03_01"
                row[11] = current_width
                row[12] = current_width
                row[24] = cond
                count_03_01 +=1
                voc_null += 1
                
                
            #VOC Null
            #One Way Street
            #ASSUMED One Lane
            elif (      
                (row[22] is None) and                                #VOC
                (row[13] is None) and
                (row[14] is None) and             #TransCAD Lanes IS NULL
                (row[10] in [1,2]) and                                #one-way
                (row[19] is not None) and
                (row[19] < param_one_way)
            ):
                
                
                current_width = 13
                #print("One Way - VOC Max: NULL, Current:{0}, Proposed: N/A Assumed 1 lane".format(current_width))
                
                cond = "03_02"
                row[11] = current_width
                row[12] = current_width
                row[20] = 1
                row[24] = cond
                count_03_02 +=1
                voc_null += 1
                
                
            #VOC Null
            #Two Way Street
            #TransCAD Lanes
            #note this most likely does not occur but was implemented to catch records if they do occur
            elif (
                (row[22] is None) and
                (row[13] is not None) and
                (row[14] is not None) and
                (row[10] == 0)
            ):
                
                current_width = (row[13] + row[14])*11
                #print("Two Way - VOC Max: NULL, Current:{0}, Proposed: N/A".format(current_width))
                
                cond = "03_03"
                row[11] = current_width
                row[12] = current_width
                row[24] = cond
                count_03_03 +=1
                voc_null += 1
                
                
            
            
            #VOC Null
            #Two Way Street
            #HPMS Lanes
            elif (
                (row[22] is None) and        #VOC is NULL
                (row[23] is not None) and  #HPMS lanes are not null
                (row[10] == 0)           #one way indicators
                 ): 

                
                current_width = (row[23]*11)+2
                #print("One Way - VOC Max: NULL, Current:{0}, Proposed: N/A Assumed 2 lanes".format(current_width))
                
                cond = "03_04_01"
                row[11] = current_width
                row[12] = current_width
                row[20] = 2
                row[24] = cond
                count_03_04_01 +=1
                voc_null += 1
                
            

            #VOC Null
            #Two Way Street
            #ASSUMED Two Lanes
            elif (((row[22] is None) and
                  (row[13] is None and row[14] is None) and
                   (row[10] == 0) and
                   (row[19] is not None) and row[19] < param_two_way)
                  ):
                
                current_width = 24
                #print("One Way - VOC Max: NULL, Current:{0}, Proposed: N/A Assumed 2 lanes".format(current_width))
                
                cond = "03_04"
                row[11] = current_width
                row[12] = current_width
                row[20] = 2
                row[24] = cond
                count_03_04 +=1
                voc_null += 1
                
                
            #VOC Null
            #Mean ROW 2020 Null
            #Street Class = A41
            #ASSUMED Two Lanes, ASSUME Modal Width (current and future) = 22'
            elif ((row[22] is None) and
                  ((row[13] is None and row[14] is None) and
                   (row[19] is None) and row[21] == 'A41')
                ):
                
                current_width = 24
                #print("A41 Assumption - VOC Max: NULL, Current:NULL, Proposed: N/A Assumed 2 lanes".format(current_width))
                
                cond = "03_05"
                row[11] = current_width
                row[12] = current_width
                row[20] = 2
                row[24] = cond
                count_03_05 +=1
                voc_null += 1
                
                                          
            cursor.updateRow(row)
            count = count + 1
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1}, {2} ".format(row[15],error,cond))
            
            
print("Done - processed {0} records, {1} errors".format(count, err_count))
print("01-01 VOC >1.0 - One Way - TransCad Lanes: {}".format(count_01_01))
print("01-02 VOC >1.0 - One Way - Assumed Lanes: {}".format(count_01_02))
print("01-03 VOC >1.0 - Two Way - TransCad Lanes: {}".format(count_01_03))
print("01-04 VOC >1.0 - Twy Way - Assumed Lanes: {}".format(count_01_04))

print("02-01 VOC <1.0 - One Way - TransCad Lanes: {}".format(count_02_01))
print("02-02 VOC <1.0 - One Way - Assumed Lanes: {}".format(count_02_02))
print("02-03 VOC <1.0 - Two Way - TransCad Lanes: {}".format(count_02_03))
print("02-04 VOC <1.0 - Two Way - Assumed Lanes: {}".format(count_02_04))

print("03-00 VOC NULL - One Way - HPMS Lanes: {}".format(count_03_00))
print("03-01 VOC NULL - One Way - TransCad Lanes: {}".format(count_03_01))
print("03-02 VOC NULL - One Way - Assumed Lanes: {}".format(count_03_02))
print("03-03 VOC NULL - Two Way - TransCad Lanes: {}".format(count_03_03))
print("03-04 VOC NULL - Two Way - Assumed Lanes: {}".format(count_03_04))
print("03-04_01 VOC NULL - Two Way - HPMS Lanes: {}".format(count_03_04_01))

print("03-05 A41 NULL -   N/A   - Assumed Lanes: {}".format(count_03_05))

print("VOC Above: {}".format(voc_above))
print("VOC Below: {}".format(voc_below))
print("VOC Null: {}".format(voc_null))
print("Modal Widths - Traffic complete.")


In [ ]:
#Conditional Re-Write2
#PARAMETERS
param_one_way = 75.0
param_two_way = 100.0

#fields to expose to update cursor
VOC_fields = ["AB_VOC_EA_1", #0
              "BA_VOC_EA_1", #1
              "AB_VOC_AM_1", #2
              "BA_VOC_AM_1", #3
              "AB_VOC_MD_1", #4
              "AB_VOC_MD_1", #5
              "AB_VOC_PM_1", #6
              "BA_VOC_PM_1", #7
              "AB_VOC_EV_1", #8
              "BA_VOC_EV_1", #9
              "ONEWAY",      #10
              "modal_width_auto_exist", #11
              "modal_width_auto_prop",  #12
              "AB_LANE",     #13
              "BA_LANE",     #14
              "SEGMENTID",   #15
              "ln_exist_num",#16
              "AB_VOC_MAX",  #17
              "BA_VOC_MAX",  #18 
              "mean_row_2020", #19
              "lanes_assumed", #20,
              "CLASS",         #21
              "VOC_MAX",    #22
              "thrulanes",   #23  HPMS Lanes
              "modal_width_auto_rule" #24
             ]

print("Update rows for Traffic widths...")
count = 0
count_01_01 = 0
count_01_02 = 0
count_01_03 = 0
count_01_04 = 0
count_02_01 = 0
count_02_02 = 0
count_02_03 = 0
count_02_04 = 0
count_03_00 = 0
count_03_01 = 0
count_03_02 = 0
count_03_03 = 0
count_03_04 = 0
count_03_04_01 = 0
count_03_05 = 0
voc_above = 0
voc_below = 0
voc_null = 0
err_count = 0


where_clause = '("VOC_max" is NOT null) AND ("VOC_Max" >= 1.0)'


with arcpy.da.UpdateCursor(modal_composite, VOC_fields, where_clause) as cursor:
    for row in cursor:
        #print("row")
        try:
            
            #CONDITIONAL GROUP 01
            
            #VOC > 1.0
            #TransCAD Lanes
            #One Way Streets
                   
            if(
                (row[13] is not None) and
                (row[14] is not None) and    #TransCAD Lanes
                (row[10] in [1,2])
            ):

                #take max of the two lane values and multiply by 11
                current_width = ((max(row[13],row[14]))*11) + 2
                expanded_width = current_width + 11
            
                #print("One Way - VOC Max:{0}, Current:{1}, Proposed: {2}".format(VOC_max,current_width,expanded_width))
                
                cond = "01_01" 
                row[11] = current_width
                row[12] = expanded_width
                row[24] = cond
                voc_above +=1 
                count_01_01 +=1
   
            
            #VOC > 1.0
            #One Way Streets
            #ASSUMED Lanes
            elif (
                (row[13] is None) and
                (row[14] is None) and             #TransCAD Lanes IS NULL
                (row[10] in [1,2]) and                              #oneway indicator
                (row[19] is not None) and
                (row[19] < param_one_way)
            ):
                
                current_width = 11
                expanded_width = 22
                #print("One Way - VOC Max:{0}, Current:{1}, Proposed: {2} Assumed 1 lane".format(VOC_max,current_width,expanded_width))
                
                cond = "01_02"
                
                row[11] = current_width
                row[12] = expanded_width
                row[20] = 1
                row[24] = cond
                
                voc_above +=1 
                count_01_02 +=1
                
                
                
            #VOC > 1.0
            #Two Way Streets
            #TransCAD Lanes
            elif (
                (row[13] is not None) and
                (row[14] is not None) and  #TransCAD Lanes
                (row[10] == 0)                                  #two-way indicator
            ):
                
                current_width = ((row[13] + row[14])*11) + 2
                expanded_width = current_width + 22
                #print("Two Way - VOC Max:{0}, Current:{1}, Proposed: {2}".format(VOC_max,current_width,expanded_width))
                
                cond = "01_03"
                row[11] = current_width
                row[12] = expanded_width
                row[24] = cond
                voc_above +=1
                count_01_03 +=1
                
                
            #VOC > 1.0
            #Two Way Streets
            #ASSUMED 2 Lanes
            elif (
                (row[13] is None) and
                (row[14] is None) and             #TransCAD Lanes IS NULL
                (row[10] == 0) and                                  #two-way indicator
                (row[19] is not None) and
                (row[19] < param_two_way)  #ROW width parameter
            ):
                
                current_width = 24
                expanded_width = 44
                #print("One Way - VOC Max:{0}, Current:{1}, Proposed: {2} Assumed 2 lanes".format(VOC_max,current_width,expanded_width))
                
                cond = "01_04"
                row[11] = current_width
                row[12] = expanded_width
                row[20] = 2
                row[24] = cond
                voc_above +=1
                count_01_04 +=1
                
                
                
            cursor.updateRow(row)
            count = count + 1
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1}, {2} ".format(row[15],error,cond))
            
            
print("Done - processed {0} records, {1} errors".format(count, err_count))
print("01-01 VOC >1.0 - One Way - TransCad Lanes: {}".format(count_01_01))
print("01-02 VOC >1.0 - One Way - Assumed Lanes: {}".format(count_01_02))
print("01-03 VOC >1.0 - Two Way - TransCad Lanes: {}".format(count_01_03))
print("01-04 VOC >1.0 - Twy Way - Assumed Lanes: {}".format(count_01_04))
            

    
    
    
    
#CONDITIONAL GROUP 02
            
    
count = 0           
where_clause = '("VOC_max" is NOT null) AND ("VOC_Max" < 1.0)'


with arcpy.da.UpdateCursor(modal_composite, VOC_fields, where_clause) as cursor:
    for row in cursor:
        #print("row")
        try:

            
            #VOC < 1.0
            #One Way Streets
            #TransCAD Lanes
            if (
                (row[13] is not None) and  #AB_Lane
                (row[14] is not None) and  #BA_Lane
                (row[10] in [1,2])
            ):
                
                current_width = ((max(row[13],row[14]))*11) + 2  
                
                #print("One Way - VOC Max <1.0, Current:{0}, Proposed: {1} - {2}-{3}".format(current_width,expanded_width, row[17], row[18]))
                
                cond = "02_01"
                row[11] = current_width
                row[12] = current_width
                row[24] = cond
                count_02_01 +=1
                voc_below += 1
                
                
            
            #VOC < 1.0
            #One Way Streets
            #ASSUMED One Lane

            elif (
                (row[13] is None) and
                (row[14] is None) and               #TransCAD Lanes IS NULL
                (row[10] in [1,2]) and                                      #one-way indicator
                (row[19] is not None) and
                (row[19] < param_one_way)#one-way width assumption parameter 
            ): 
                
                current_width = 13
                
                #print("One Way - VOC Max:{0}, Current:{1}, Proposed: {2} Assumed 1 lane".format(VOC_max,current_width,expanded_width))
                
                cond = "02_02"
                row[11] = current_width
                row[12] = current_width
                row[20] = 1
                row[24] = cond
                count_02_02 +=1
                voc_below += 1
                
                
            #VOC < 1.0
            #Two Way Streets
            #TransCAD Lanes
            elif (
                (row[13] is not None) and
                (row[14] is not None) and   #TransCAD Lanes     
                (row[10] == 0)
            ):
                
                current_width = ((row[13] + row[14])*11)+2
                
                #print("Two Way - VOC Max <1.0, Current:{0}, Proposed: {1}".format(current_width,expanded_width))
                
                cond = "02_03"
                row[11] = current_width
                row[12] = current_width
                row[24] = cond
                count_02_03 +=1
                voc_below += 1
                
                
            #VOC < 1.0
            #Two Way Streets
            #ASSUMED Two Lanes
            elif (
                (row[13] is None) and
                (row[14] is None) and            #TransCAD Lanes
                (row[10] == 0) and                                 #two-way indicator
                (row[19] is not None) and
                (row[19] < param_two_way) #ROW parameter
            ):
                
                current_width = 24
                
                #print("One Way - VOC Max:{0}, Current:{1}, Proposed: {2} Assumed 2 lanes".format(VOC_max,current_width,expanded_width))
                
                cond = "02_04"
                row[11] = current_width
                row[12] = current_width
                row[20] = 2
                row[24] = cond
                count_02_04 +=1
                voc_below += 1
                
            cursor.updateRow(row)
            count = count + 1
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1}, {2} ".format(row[15],error,cond))
            
            
            
print("Done - processed {0} records, {1} errors".format(count, err_count))            
print("02-01 VOC <1.0 - One Way - TransCad Lanes: {}".format(count_02_01))
print("02-02 VOC <1.0 - One Way - Assumed Lanes: {}".format(count_02_02))
print("02-03 VOC <1.0 - Two Way - TransCad Lanes: {}".format(count_02_03))
print("02-04 VOC <1.0 - Two Way - Assumed Lanes: {}".format(count_02_04))

                
  

#CONDITIONAL GROUP 03 - VOC NULL

count = 0                
where_clause = '("VOC_max" is null)'
with arcpy.da.UpdateCursor(modal_composite, VOC_fields, where_clause) as cursor:
    for row in cursor:
        #print("row")
        try:                
                        
            #One Way Streets
            #HPMS Lanes
            if (
                (row[23] is not None) and  #HPMS lanes are not null
                (row[10] in [1,2] )
            ):
 
                current_width = (row[23]*11)+2
                
                #print("One Way - VOC Max:{0}, Current:{1}, Proposed: {2} Assumed 1 lane".format(VOC_max,current_width,expanded_width))
                cond = "03_00"
                row[11] = current_width
                row[12] = current_width
                row[20] = 1
                row[24] = cond
                
                voc_above +=1 
                count_03_00 +=1
                
            
            #VOC Null
            #One Way Street
            #TransCAD Lanes
            #note this most likely does not occur but was implemented to catch records if they do occur
            elif (
                (row[13] is not None) and 
                (row[14] is not None) and
                (row[10] in [1,2])
            ):
        
                current_width = (max(row[13],row[14])) * 11
                #print("One Way - VOC Max: NULL, Current:{0}, Proposed: N/A".format(current_width))
                
                cond = "03_01"
                row[11] = current_width
                row[12] = current_width
                row[24] = cond
                count_03_01 +=1
                voc_null += 1
                
                
            #VOC Null
            #One Way Street
            #ASSUMED One Lane
            elif (      
                (row[13] is None) and
                (row[14] is None) and             #TransCAD Lanes IS NULL
                (row[10] in [1,2]) and                                #one-way
                (row[19] is not None) and
                (row[19] < param_one_way)
            ):
                
                
                current_width = 13
                #print("One Way - VOC Max: NULL, Current:{0}, Proposed: N/A Assumed 1 lane".format(current_width))
                
                cond = "03_02"
                row[11] = current_width
                row[12] = current_width
                row[20] = 1
                row[24] = cond
                count_03_02 +=1
                voc_null += 1
                
                
            #VOC Null
            #Two Way Street
            #TransCAD Lanes
            #note this most likely does not occur but was implemented to catch records if they do occur
            elif (
                (row[13] is not None) and
                (row[14] is not None) and
                (row[10] == 0)
            ):
                
                current_width = (row[13] + row[14])*11
                #print("Two Way - VOC Max: NULL, Current:{0}, Proposed: N/A".format(current_width))
                
                cond = "03_03"
                row[11] = current_width
                row[12] = current_width
                row[24] = cond
                count_03_03 +=1
                voc_null += 1
                
                
            
            
            #VOC Null
            #Two Way Street
            #HPMS Lanes
            elif (
                (row[23] is not None) and  #HPMS lanes are not null
                (row[10] == 0)           #one way indicators
                 ): 

                
                current_width = (row[23]*11)+2
                #print("One Way - VOC Max: NULL, Current:{0}, Proposed: N/A Assumed 2 lanes".format(current_width))
                
                cond = "03_04_01"
                row[11] = current_width
                row[12] = current_width
                row[20] = 2
                row[24] = cond
                count_03_04_01 +=1
                voc_null += 1
                
            

            #VOC Null
            #Two Way Street
            #ASSUMED Two Lanes
            elif (
                (row[13] is None) and
                (row[14] is None) and
                (row[10] == 0) and
                (row[19] is not None) and
                (row[19] < param_two_way)
                  ):
                
                current_width = 24
                #print("One Way - VOC Max: NULL, Current:{0}, Proposed: N/A Assumed 2 lanes".format(current_width))
                
                cond = "03_04"
                row[11] = current_width
                row[12] = current_width
                row[20] = 2
                row[24] = cond
                count_03_04 +=1
                voc_null += 1
                
                
            #VOC Null
            #Mean ROW 2020 Null
            #Street Class = A41
            #ASSUMED Two Lanes, ASSUME Modal Width (current and future) = 22'
            elif (
                  (row[13] is None) and
                  (row[14] is None) and
                  (row[19] is None) and
                  (row[21] == 'A41')
                ):
                
                current_width = 24
                #print("A41 Assumption - VOC Max: NULL, Current:NULL, Proposed: N/A Assumed 2 lanes".format(current_width))
                
                cond = "03_05"
                row[11] = current_width
                row[12] = current_width
                row[20] = 2
                row[24] = cond
                count_03_05 +=1
                voc_null += 1
                
                                          
            cursor.updateRow(row)
            count = count + 1
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1}, {2} ".format(row[15],error,cond))
            
            



print("Done - processed {0} records, {1} errors".format(count, err_count))
print("03-00 VOC NULL - One Way - HPMS Lanes: {}".format(count_03_00))
print("03-01 VOC NULL - One Way - TransCad Lanes: {}".format(count_03_01))
print("03-02 VOC NULL - One Way - Assumed Lanes: {}".format(count_03_02))
print("03-03 VOC NULL - Two Way - TransCad Lanes: {}".format(count_03_03))
print("03-04 VOC NULL - Two Way - Assumed Lanes: {}".format(count_03_04))
print("03-04_01 VOC NULL - Two Way - HPMS Lanes: {}".format(count_03_04_01))

print("03-05 A41 NULL -   N/A   - Assumed Lanes: {}".format(count_03_05))

print("VOC Above: {}".format(voc_above))
print("VOC Below: {}".format(voc_below))
print("VOC Null: {}".format(voc_null))
print("Modal Widths - Traffic complete.")





##Add 2' width to all auto widths on streets that contain medians (additional outside lane)
count = 0
where_clause = 'NOT ("median_type" IS NULL) AND NOT ("modal_width_auto_rule" IS NULL)'

with arcpy.da.UpdateCursor(modal_composite, VOC_fields, where_clause) as cursor:
    for row in cursor:
        #print("row")
        try:
            row[11] = row[11]+2
            row[12] = row[12]+2
            row[24] = row[24] + " MEDIAN"
            count += 1
        except ValueError as error:
            print(error)
            

            
print("median buffers calculated for {} items".format(count))







In [ ]:
#Create DF for CSDM analysis
df = pd.DataFrame.spatial.from_featureclass(modal_composite)

df_fields = df.columns.values.tolist()

df_fields.sort()

# for i in df_fields:
#      print(i)

In [ ]:
#fields = ["SEGMENTID","FULLNAME","ONEWAY","AB_VOC_MAX","BA_VOC_MAX","VOC_max","AB_LANE","BA_LANE","ln_exist_num","thrulanes","TPN_OID","NewTier","mean_row_2020","modal_width_auto_exist","modal_width_auto_prop"]

#fields = ["SEGMENTID","FULLNAME","CLASS","ONEWAY","AB_VOC_MAX","BA_VOC_MAX","VOC_max","AB_LANE","BA_LANE","ln_exist_num","thrulanes","TPN_OID","NewTier","mean_row_2020","modal_width_auto_rule","modal_width_auto_exist","modal_width_auto_prop"]

fields = ["SEGMENTID","FULLNAME","CLASS","ONEWAY","NewTier","NBRightP_7","SBLeftPe_6","mean_row_2020","modal_width_park_exist","modal_width_ped_exist","modal_width_ped_prop","modal_width_ped_furn"]



filt = (df["modal_width_ped_exist"]>0)

#two way streets, null lane count, row > 100
filt2 = ((df["AB_LANE"].isnull()) &
         (df["BA_LANE"].isnull()) &
         (df["ONEWAY"] == 0) &
         (df["mean_row_2020"]>100))
         
#one way streets, null lane count, row > 75
filt3 = ((df["AB_LANE"].isnull()) &
         (df["BA_LANE"].isnull()) &
         (df["ONEWAY"].isin([1,2])) &
         (df["mean_row_2020"]>75))

filt4 = (df["CLASS"] == 'A41')

filt5 = ((~df["TPN_OID"].isnull()) &
         (df["NewTier"] == 4))

#01-01 - VOC >1.0 - One Way - TransCAD Lanes
filt_cond_1_1 = (~(df["AB_LANE"].isnull()) &
         ~(df["BA_LANE"].isnull()) &
         (df["VOC_max"] >1.0 ) &
         df["ONEWAY"].isin([1,2]))

#01-02 - VOC >1.0 - One Way - Assumed Lanes
filt_cond_1_2 = ((df["AB_LANE"].isnull()) &
         (df["BA_LANE"].isnull()) &
         (df["VOC_max"] >1.0 ) &
         (df["mean_row_2020"] < param_one_way ) &
         df["ONEWAY"].isin([1,2]))

#01-03 - VOC >1.0 - Two-Way - TransCAD Lanes
filt_cond_1_3 = (~(df["AB_LANE"].isnull()) &
         ~(df["BA_LANE"].isnull()) &
         (df["VOC_max"] >1.0 ) &
         df["ONEWAY"].isin([0]))

#01-04 - VOC >1.0 - Two-Way - Assumed Lanes
filt_cond_1_4 = ((df["AB_LANE"].isnull()) &
         (df["BA_LANE"].isnull()) &
         (df["VOC_max"] >1.0 ) &
         (df["mean_row_2020"] < param_two_way ) &
         df["ONEWAY"].isin([0]))

#02-01 - VOC <1.0 - One Way - TransCAD Lanes
filt_cond_2_1 = (~(df["AB_LANE"].isnull()) &
         ~(df["BA_LANE"].isnull()) &
         (df["VOC_max"] <1.0 ) &
         df["ONEWAY"].isin([1,2]))

#02-02 - VOC <1.0 - One Way - Assumed Lanes
filt_cond_2_2 = ((df["AB_LANE"].isnull()) &
         (df["BA_LANE"].isnull()) &
         (df["VOC_max"] <1.0 ) &
         (df["mean_row_2020"] < 75 ) &
         df["ONEWAY"].isin([1,2]))

#02-03 - VOC <1.0 - Two-Way - TransCAD Lanes
filt_cond_2_3 = (~(df["AB_LANE"].isnull()) &
         ~(df["BA_LANE"].isnull()) &
         (df["VOC_max"] <1.0 ) &
         df["ONEWAY"].isin([0]))

#02-04 - VOC <1.0 - Two-Way - Assumed Lanes
filt_cond_2_4 = ((df["AB_LANE"].isnull()) &
         (df["BA_LANE"].isnull()) &
         (df["VOC_max"] <1.0 ) &
         (df["mean_row_2020"] < param_two_way ) &
         df["ONEWAY"].isin([0]))

#03-00 - VOC NULL - Two-Way - HPMS Lanes
filt_cond_3_00 = (
    (df["VOC_max"].isnull()) &
    ~(df["thrulanes"].isnull()) &
    (df["ONEWAY"].isin([1,2])))


#03-01 - VOC NULL - One-Way - TransCAD Lanes
filt_cond_3_01 = (
    (df["VOC_max"].isnull()) &
    ~(df["AB_LANE"].isnull()) &
    (df["ONEWAY"].isin([1,2]))
)

#03-02 - VOC NULL - One-Way - Assumed Lanes
filt_cond_3_02 = (
    (df["VOC_max"].isnull()) &
    (df["AB_LANE"].isnull()) &
    (df["thrulanes"].isnull()) &
    (df["mean_row_2020"] < param_one_way) &
    (df["ONEWAY"].isin([1,2]))
)

#03-03 - VOC NULL - Two-Way - HPMS
filt_cond_3_03 = (
    (df["VOC_max"].isnull()) &
    ~(df["thrulanes"].isnull()) &
    (df["ONEWAY"] == 0)
)

#03-04 - VOC NULL - Two-Way - Assumed Lanes
filt_cond_3_04 = (
    (df["VOC_max"].isnull()) &
    (df["AB_LANE"].isnull()) &
    (df["BA_LANE"].isnull()) &
    ~(df["mean_row_2020"].isnull()) &
    (df["mean_row_2020"] < param_two_way) &
    (df["ONEWAY"] == 0)
    
)

#03-05 - VOC NULL - ROW NULL - Class = A41
filt_cond_3_05 = (
    (df["VOC_max"].isnull()) &
    (df["AB_LANE"].isnull()) &
    (df["BA_LANE"].isnull()) &
    (df["mean_row_2020"].isnull()) &
    (df["CLASS"] == 'A41')
    
)



filt_lanes = (
    (df["AB_LANE"].isnull()) &
    (df["BA_LANE"].isnull()) &
    ~(df["thrulanes"].isnull()))


filt_NewTier = (
    ~(df["NewTier"].isnull()))

#misses
filt_cond_miss = (
    (df["modal_width_auto_rule"].isnull())
)




#03-01 VOC NULL - One Way - TransCad Lanes
#03-02 VOC NULL - One Way - Assumed Lanes
#03-03 VOC NULL - Two Way - TransCad Lanes
#03-04 VOC NULL - Two Way - Assumed Lanes
#03-05 A41 NULL -   N/A   - Assumed Lanes


pd.set_option('display.max_rows', 250)


df.loc[filt_NewTier,fields].sort_values(by="FULLNAME").head(250)

In [ ]:
#QC Auto Conditionals

print("Conditional 01_01 check..")
print(df.loc[filt_cond_1_1,fields].sort_values(by=["AB_LANE","BA_LANE"],ascending=False).shape[0])
print(df.loc[(df["modal_width_auto_rule"]=='01_01'),fields].sort_values(by=["AB_LANE","BA_LANE"],ascending=False).shape[0])
print("\n")

print("Conditional 01_02 check..")
print(df.loc[filt_cond_1_2,fields].sort_values(by=["AB_LANE","BA_LANE"],ascending=False).shape[0])
print(df.loc[(df["modal_width_auto_rule"]=='01_02'),fields].sort_values(by=["AB_LANE","BA_LANE"],ascending=False).shape[0])
print("\n")

print("Conditional 01_03 check..")
print(df.loc[filt_cond_1_3,fields].sort_values(by=["AB_LANE","BA_LANE"],ascending=False).shape[0])
print(df.loc[(df["modal_width_auto_rule"]=='01_03'),fields].sort_values(by=["AB_LANE","BA_LANE"],ascending=False).shape[0])
print("\n")

print("Conditional 01_04 check..")
print(df.loc[filt_cond_1_4,fields].sort_values(by=["AB_LANE","BA_LANE"],ascending=False).shape[0])
print(df.loc[(df["modal_width_auto_rule"]=='01_04'),fields].sort_values(by=["AB_LANE","BA_LANE"],ascending=False).shape[0])
print("\n")

###

print("Conditional 02_01 check..")
print(df.loc[filt_cond_2_1,fields].sort_values(by=["AB_LANE","BA_LANE"],ascending=False).shape[0])
print(df.loc[(df["modal_width_auto_rule"]=='02_01'),fields].sort_values(by=["AB_LANE","BA_LANE"],ascending=False).shape[0])
print("\n")

print("Conditional 02_02 check..")
print(df.loc[filt_cond_2_2,fields].sort_values(by=["AB_LANE","BA_LANE"],ascending=False).shape[0])
print(df.loc[(df["modal_width_auto_rule"]=='02_02'),fields].sort_values(by=["AB_LANE","BA_LANE"],ascending=False).shape[0])
print("\n")

print("Conditional 02_03 check..")
print(df.loc[filt_cond_2_3,fields].sort_values(by=["AB_LANE","BA_LANE"],ascending=False).shape[0])
print(df.loc[(df["modal_width_auto_rule"]=='02_03'),fields].sort_values(by=["AB_LANE","BA_LANE"],ascending=False).shape[0])
print("\n")

print("Conditional 02_04 check..")
print(df.loc[filt_cond_2_4,fields].sort_values(by=["AB_LANE","BA_LANE"],ascending=False).shape[0])
print(df.loc[(df["modal_width_auto_rule"]=='02_04'),fields].sort_values(by=["AB_LANE","BA_LANE"],ascending=False).shape[0])
print("\n")

###
print("Conditional 03_00 check..")
print(df.loc[filt_cond_3_00,fields].sort_values(by=["AB_LANE","BA_LANE"],ascending=False).shape[0])
print(df.loc[(df["modal_width_auto_rule"]=='03_00'),fields].sort_values(by=["AB_LANE","BA_LANE"],ascending=False).shape[0])
print("\n")

print("Conditional 03_01 check..")
print(df.loc[filt_cond_3_01,fields].sort_values(by=["AB_LANE","BA_LANE"],ascending=False).shape[0])
print(df.loc[(df["modal_width_auto_rule"]=='03_01'),fields].sort_values(by=["AB_LANE","BA_LANE"],ascending=False).shape[0])
print("\n")

print("Conditional 03_02 check..")
print(df.loc[filt_cond_3_02,fields].sort_values(by=["AB_LANE","BA_LANE"],ascending=False).shape[0])
print(df.loc[(df["modal_width_auto_rule"]=='03_02'),fields].sort_values(by=["AB_LANE","BA_LANE"],ascending=False).shape[0])
print("\n")

print("Conditional 03_03 check..")
print(df.loc[filt_cond_3_03,fields].sort_values(by=["AB_LANE","BA_LANE"],ascending=False).shape[0])
print(df.loc[(df["modal_width_auto_rule"]=='03_03'),fields].sort_values(by=["AB_LANE","BA_LANE"],ascending=False).shape[0])
print("\n")

print("Conditional 03_04 check..")
print(df.loc[filt_cond_3_04,fields].sort_values(by=["AB_LANE","BA_LANE"],ascending=False).shape[0])
print(df.loc[(df["modal_width_auto_rule"]=='03_04'),fields].sort_values(by=["AB_LANE","BA_LANE"],ascending=False).shape[0])
print("\n")

print("Conditional 03_05 check..")
print(df.loc[filt_cond_3_05,fields].sort_values(by=["AB_LANE","BA_LANE"],ascending=False).shape[0])
print(df.loc[(df["modal_width_auto_rule"]=='03_05'),fields].sort_values(by=["AB_LANE","BA_LANE"],ascending=False).shape[0])
print("\n")

In [ ]:
lanes = 3
lane_width = (lanes-2)*10 + 22

expanded_width = (lanes-1)*10 + 22

print(lane_width)
print(expanded_width)

### Calculate Modal Widths for Bike Facilities


- Existing width is calculated into `modal_width_bike_exist` and `modal_width_bike_prop`

- Proposed width is then updated into `modal_width_bike_prop` based on proposed and redevelopment bike plan attributes.

- If a give segment has a value for existing bike facilites, but no values for proposed facilites, existing width is calculted into proposed width.


**Assumptions, Notes, and Parameters**
1. "Protected Bike Lane", "Buffered Bike Lane" = 10'
2. "Bike Lane", "Climbing Lane", "Shoulder Bikeway" = 7'


In [ ]:
#fields to expose to update cursor for bike facilities
fields = ["Fac_Type_BE",            #0
          "modal_width_bike_exist", #1
          "Fac_Type_BP",            #2
          "Fac_Type_BR",            #3
          "modal_width_bike_prop"]  #4

#update bike existing width columns
print("Update rows for Bike widths..")
with arcpy.da.UpdateCursor(modal_composite_05a,fields,'NOT ("Fac_Type_BE" IS NULL)') as cursor:
    for row in cursor:
        #print("row")
        try:
            #bike existing calculations
            if row[0] == "Protected Bike Lane" or row[0] == "Buffered Bike Lane":
                row[1] = 12
                row[4] = 12
                    
            elif (row[0] == "Bike Lane" or
                  row[0] == "Climbing Lane" or
                  row[0] == "Shoulder Bikeway" or
                  row[0] == "Shared Use Path" or
                  row[0] == "Shared Roadway"):
                row[1] = 7
                row[4] = 7
                    
            cursor.updateRow(row)
        except ValueError as error:
            print(error)            
print("bike existing width calculated\n")

#perform second pass w/ update cursor for bike proposed features (will over write previous calcs if there is an improvement)
#update bike proposed width columns
print("Update rows for Bike widths..")
with arcpy.da.UpdateCursor(modal_composite_05a,fields,'NOT ("Fac_Type_BP" IS NULL AND "Fac_Type_BR" IS NULL)') as cursor:
    for row in cursor:
        #print("row")
        try:
            #bike proposed & redev calculations
            if (row[2] == "Protected Bike Lane" or
                  row[2] == "Buffered Bike Lane" or
                  row[3] == "Protected Bike Lane" or
                  row[3] == "Buffered Bike Lane"):
                row[4] = 12
                
            elif (row[2] == "Bike Lane" or
                  row[2] == "Climbing Lane" or
                  row[2] == "Shoulder Bikeway" or
                  row[2] == "Shared Use Path" or
                  row[2] == "Shared Roadway" or
                  row[3] == "Bike Lane" or
                  row[3] == "Climbing Lane" or
                  row[3] == "Shoulder Bikeway" or
                  row[3] == "Shared Use Path" or
                  row[3] == "Shared Roadway"):
                row[4] = 7
                
            cursor.updateRow(row)
        except ValueError as error:
            print(error)            
print("bike proposed width calculated\n")
print("Modal Widths - Bike complete.")

In [ ]:
#data analysis for medians
fields = ["SEGMENTID",
          "FULLNAME",
          "mean_row_2020",
          "NBRightP_1",
          "SBLeftPe_1",
          "FID_Ped_Improve_temp",
          "FID_Ped_Add_temp",
          "modal_width_ped_exist",
          "median_type",
          "median_width"]

fields_median = ["SEGMENTID",
          "FULLNAME",
          "mean_row_2020",
          "modal_width_medians",
          "median_type",
          "median_width"]

#two way streets, null lane count, row > 100
filt2 = ((~sdf["FID_Ped_Improve_temp"].isnull()) |
         (~sdf["FID_Ped_Add_temp"].isnull())
        )


filt3 = ((sdf["FID_Ped_Improve_temp"].isnull()) &
         (~sdf["FID_Ped_Add_temp"].isnull()) &
         (sdf["NBRightP_1"] == 0) &
         (sdf["SBLeftPe_1"] == 0)
        )
         
#one way streets, null lane count, row > 75
filt4 = ((sdf["AB_LANE"].isnull()) &
         (sdf["BA_LANE"].isnull()) &
         (sdf["ONEWAY"].isin([1,2])) &
         (sdf["mean_row_2020"]>75)
        )

#two way streets, null lane count, row > 100
filt_median = ~sdf["median_type"].isnull()

#pd.set_option('display.max_rows', 250)
sdf.loc[filt_median,fields_median].sort_values(by="FULLNAME")

In [ ]:
sdf2.loc[sdf2["median_type"]== "R","median_type"] = "Raised with Curb"
sdf2.loc[sdf2["median_type"]== "F","median_type"] = "Flush Two-Way Left Turn"
sdf2.loc[sdf2["median_type"]== "C","median_type"] = "Composite"
sdf2.loc[sdf2["median_type"]== "FB","median_type"] = "Flush with Barrier"

sdf2.loc[sdf2["median_width"]== "A","median_width"] = 5
sdf2.loc[sdf2["median_width"]== "B","median_width"] = 11
sdf2.loc[sdf2["median_width"]== "C","median_width"] = 17
sdf2.loc[sdf2["median_width"]== "D","median_width"] = 22

sdf2_filt = ((~sdf2["median_type"].isnull()) &
             (~sdf2["median_type"].isin(["Raised", "Flush", "Composite", "Flush with Barrier"]))
            )

sdf2.loc[~sdf2["median_type"].isnull(),["SEGMENTID", "median_type", "median_width"]]

In [ ]:
sdf_fields

### Calculate Modal Widths for Medians

- Median data was manually classified by reviewing street segments where `mean_row_2020 > 75`, then estimaing median type and median width using aerial imagery.
- Width estimates were then mapped into `modal_width_medians` (see assumptions below)

**Assumptions, Notes, and Parameters**
1. Median width esitmations were mapped to modal with values as follows:

|Median Width Estimate| Modal Width Value  |
|:--|:--|
| < 5' |5'  |
| 5' - 10' |11' (increase by 1' to match avg. lane width)  |
| 15' - 20' |17'  |
| >20' |22'  |


In [ ]:
#Calculate Modal Width for Medians

# "A","median_width"] = 5
# "B","median_width"] = 11
# "C","median_width"] = 17
# "D","median_width"] = 22

#fields to expose to update cursor
fields = ["median_type",         #0
          "median_width",        #1
          "modal_width_medians"] #2

#update median width columns
print("Update rows for Medians widths..")
count = 0
with arcpy.da.UpdateCursor(modal_composite,fields,'NOT ("median_type" IS NULL)') as cursor:
    for row in cursor:
        #print("row")
        
        # 6/16/2021 - increase all values by 2
        
        try:
            if row[1] == "A":
                row[2] = 7
                count +=1
            elif row[1] == "B":
                row[2] = 13
                count +=1
            elif row[1] == "C":
                row[2] = 19
                count +=1
            elif row[1] == "D":
                row[2] = 14
                count +=1
                    
            cursor.updateRow(row)
            
        except ValueError as error:
            print(error)
            
print("median widths calculated for {} items".format(count))
print("Modal Widths - Medians complete.")

### Calculate Modal Widths for on-street Parking segments

- Existing width of 7' is calculated into `modal_width_park_exist` and `modal_width_park_prop` for the following segment types:
    - Roadway Buffer = "Yes" `NBRightP_7 == 1 OR SBLeftPed_6 == 1`
    - Pedestrian Zone Intrusion = "Yes" `NBRightP_4 == 1 or SBLeftPed_4`
    
- If a segment is not listed in the Pedestrian Plan Sidewalk inventory, and it's street type is residential (A41), and mean_row_2020 - modal_width_auto_exist > 22, on street parking is assumed on both sides.




**Assumptions, Notes, and Parameters**
1. Existing facilites from Pedestrian Sidewalk Inventory with data in "Pedestrian Zone Width" field is mapped as follows:
  
|Pedestrian Zone Width| Modal Width Value  |
|:--|:--|
| "Less than 5' " |4'  |
| "5' to 10' " |8'  |
| "Greater than 10' " |12'  |

2. Width values for "NB" and "SB" from Pedestrian Sidewalk Inventory are summed.
3. Street segments listed in "Candidate Walkways", and "Candidate Upgrades" datasets are assumed to be 6' sidewalk on both sides, (12' total)
4. Street segmnents listed in Pedestrian Sidewalk Inventory having no facilites, but are not listed in "Candidate Walkways", or "Candidate Upgrades" and are PLACEHOLDER are assumed to be upgraded to one 6' sidewalk. 6' total.


In [ ]:
fields = arcpy.ListFields(modal_composite)

for field in fields:
    if field.name == "modal_width_ped_exist":
        print(field.type)

In [ ]:
#calculate modal width for street segements with on-street parking facilities


#expose fields to update cursor
parking_fields = ["SEGMENTID",              #0 - SegmentID
                  "NBRightP_7",             #1 - Road Buffer (NB)
                  "SBLeftPe_6",             #2 - Road Buffer (SB)
                  "NBRightP_4",             #3 - Ped Intrusion (NB)
                  "SBLeftPe_4",             #4 - Ped Intrusion (SB)
                  "modal_width_park_exist", #5 - Modal Width - Park (Existing)
                  "modal_width_park_prop",  #6 - Modal Width - Park (Proposed)
                  "mean_row_2020",          #7 - Mean ROW Analysis
                  "modal_width_auto_exist", #8 - Modal Width - Auto (Existing)
                  "CLASS"]                  #9 - Street Class (tiger)

park_ped_count = 0
park_assume_count = 0
no_can_park_count = 0
err_count = 0

param = 22

print("Update rows for Parking widths..")

#calculate parking width for segments based on ped inventory facilities for NB road side
with arcpy.da.UpdateCursor(modal_composite, parking_fields,'"NBRightP_7" IS NOT NULL') as cursor:
    for row in cursor:
        #print("row")
        try:
            if((row[1] == 1) or (row[3] == 1)):
                
                row[5] = 8
                row[6] = 8
                
               
            elif(row[1] == 0 and row[3] == 0):
                row[5] = 0
                row[6] = 0  
                
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
               
               
#calculate parking width for segments based on ped inventory facilities for SB road side
with arcpy.da.UpdateCursor(modal_composite, parking_fields,'"SBLeftPe_6" IS NOT NULL') as cursor:
    for row in cursor:
        #print("row")
        try:
            if((row[2] == 1) or (row[4] == 1)):
                row[5] = row[5]+8
                row[6] = row[5]+8
                
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
               

            

#calculate parking width for segments not listed in ped inventory facilities
with arcpy.da.UpdateCursor(modal_composite, parking_fields,'"NBRightP_7" IS NULL AND "SBLeftPe_6" IS NULL AND "CLASS" = \'A41\'') as cursor:
    for row in cursor:
        #print("row")
        try:
            if (row[7] is not None) and (row[8] is not None) and ((row[7] - row[8]) > param):
                row[5] = 8
                row[6] = 8
                park_assume_count +=1
                
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            
            
            
#calculate parking width 0 for all others
with arcpy.da.UpdateCursor(modal_composite, parking_fields,'"modal_width_park_exist" IS NULL') as cursor:
    for row in cursor:
        #print("row")
        try:
            row[5] = 0
            row[6] = 0
            no_can_park_count +=1
                
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            
            
print("Parking widths (ped) calculated for {0} items".format(park_ped_count))
print("Parking widths (assumed) calculated for {0} items".format(park_assume_count))
print("Parking widths (0) calculated for {0} items".format(no_can_park_count))

print("Errors (total) for {0} items".format(err_count))
print("Modal Widths - Parking complete.")

### Calculate Modal Widths for Ped Facilities

- Existing width is calculated into `modal_width_ped_exist` and `modal_width_ped_prop`

- Proposed width `modal_width_ped_prop` is then updated if a segment had no ped facilites, was not listed in ped facilites, or was listed in "Candidate Walkways" or "Candidate Upgrades", and the proposed width was greater than the existing width. (see assumptions below)


**Assumptions, Notes, and Parameters**
1. Existing facilites from Pedestrian Sidewalk Inventory with data in "Pedestrian Zone Width" field is mapped as follows:
  
|Pedestrian Zone Width| Modal Width Value  |
|:--|:--|
| "Less than 5' " |4'  |
| "5' to 10' " |8'  |
| "Greater than 10' " |12'  |

2. Width values for "NB" and "SB" from Pedestrian Sidewalk Inventory are summed.
3. Any segment listed in `NewTier` == 4 is set to 8' if oneway, 16' if two-way for future widths.
3. The following steet segments types are set to 12' for `modal_width_ped_prop`
    1. Any street segment listed in "Candidate Walkways", and "Candidate Upgrades" datasets
    2. Any street segment listed in Pedestrian Sidewalk Inventory as having no facilites, but are not listed in "Candidate Walkways" or "Candidate Upgrades"
    3. Any street segment NOT listed in Pedestrian Sidewalk Inventory, AND not listed in "Candidate Walkways", AND not listed in "Candidate Upgrades", with `CLASS = A41`


In [ ]:
#add field to hold furniture zone width

#store field, field alias in dict
new_fields = {
              "modal_width_ped_furn": "Modal Width - Ped (Furn)"
}

#loop through dict and add fields
for key,value in new_fields.items():
    print("Adding field {0}".format(key))
    arcpy.AddField_management(modal_composite,field_name=key,field_type="SHORT", field_alias = value)
    

          
print("Field additions complete")



In [ ]:
#calculate modal width for ped facilities

#assumptions based on ped data dictionary values for PED ZONE WIDTH
# greater than 10' = 12' DOMAIN 04
# 5'-10' (clear and obstruced types) = 8' DOMAIN 03, DOMAIN 02
# less than 5' = 4' DOMAIN 01

p1 = 12 #DOMAIN VALUE 4
p2 = 8  #DOMAIN VALUE 3 and 2
p3 = 4  #DOMAIN VALUE 1

#facilities combinations
#24' total
#20' total
#16' total
#12' total
#8' total
#4' total
# none on either side

#existing ped facilities

#expose fields to update cursor
ped_e_fields = ["SEGMENTID",              #0 - SegmentID
                "NBRightP_1",             #1 - Pedestrian Zone Width (NB)
                "SBLeftPe_1",             #2 - Pedestiran Zone Width (SB)
                "modal_width_ped_exist",  #3 - Modal Width - Ped (Existing)
                "modal_width_ped_prop",   #4 - Modal Width - Ped (Proposed)
                "mean_row_2020",          #5 - Mean ROW Analysis
                "FID_Ped_Improve_temp",   #6 - Ped Improvements
                "FID_Ped_Add_temp",       #7 - Ped Additions
                "CLASS",                  #8 - Street Class (tiger)
                "NewTier",                #9 - Bus Transit - New Proposed Tier
                "ONEWAY",                 #10 - Oneway indicator
                "NBRightP_7",             #11 - Pedestrian Buffer Roadway Buffer (NB)
                "SBLeftPe_6",             #12 - Pedestrian Buffer Roadway Buffer (SB)
                "Fac_Type_BE",            #13 - Bike - Existing Type
                "Fac_Type_BP",            #14 - Bike - Proposed Type
                "Fac_Type_BR",            #15 - Bike - ReDev Type
                "modal_width_park_exist", #16 - Modal Width - Park (Existing)
                "modal_width_park_prop",  #17 - Modal Width - Park (Proposed)
                "modal_width_ped_furn"]    #18 - Modal Width - Ped (Furniture Zone)

                         


count = 0
err_count = 0

print("Update rows with Ped Zone widths..")

with arcpy.da.UpdateCursor(modal_composite, ped_e_fields) as cursor:
# with arcpy.da.UpdateCursor(modal_composite_05a, ped_e_fields,'NOT ("NBRightP_1" IS NULL OR "SBLeftPe_1" IS NULL)') as cursor:
    for row in cursor:
        #print("row")
        try:
            #24' total
            count +=1
            if row[1] == 4 and row[2] == 4:
                width = p1*2
                row[3] = width
                row[4] = width
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))
        
            #20' total
            elif ((row[1] == 4 and row[2] in [2,3])or
                  (row[1] in [2,3] and row[2] == 4)):
                width = p1+p2
                row[3] = width
                row[4] = width
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))

            #16' total
            elif ((row[1] in [2,3]) and (row[2] in [2,3])):
                width = p2*2
                row[3] = width
                row[4] = width
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))

            #12' total
            elif ((row[1] == 1 and (row[2] in [2,3]))or
                  ((row[1] in [2,3]) and row[2] == 1)):
                width = p2+p3
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))
            
            #8' total
            elif row[1] == 1 and row[2] == 1:
                width = p3*2
                row[3] = width
                row[4] = 12
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))

            #4' total
            elif ((row[1] == 1 and row[2] == 0) or
                  (row[1] == 0 and row[2] == 1)):
                width = p3
                row[3] = width
                row[4] = 12
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))


            # none on either side, set future width to 12'
            elif (row[1] == 0 and row[2] == 0):
                width = 0
                row[3] = width
                row[4] = 12
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))
                  
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))

print("Done - processed {0} records, {1} errors".format(count, err_count))




#add cursor pass for furniture zone

print("Update rows by adding Ped Furniture widths..")

with arcpy.da.UpdateCursor(modal_composite, ped_e_fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            
            #12' total
            count +=1
            if row[11] == 3 and row[12] == 3:

                
                row[3] += 12
                row[4] += 12
                row[18] = 12
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))
        
            #10' total
            elif ((row[11] == 3 and row[12] == 2)or
                  (row[11] == 2 and row[12] == 3)):
                
                row[3] += 10
                row[4] += 10
                row[18] = 10
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))
                
            #8' total
            elif (row[11] == 2 and row[12] == 2):
         
                row[3] += 8
                row[4] += 8
                row[18] = 8
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))
                
            #4' total
            elif ((row[11] == 2 and row[12] == 1)or
                  (row[11] == 1 and row[12] == 2)):
        
                row[3] += 4
                row[4] += 4
                row[18] = 4
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))
                
           #0' fill for all others
            elif ((row[11] is None) or
                  (row[12] is None)):
        
                row[18] = 0
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))


                  
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))

print("Done - processed {0} records, {1} errors".format(count, err_count))
                

print("Updating rows for NewTier values...")         
            
#proposed bus transit New tiers
count = 0
err_count = 0

with arcpy.da.UpdateCursor(modal_composite, ped_e_fields,'"NewTier" = 4 OR "NewTier" = 3 OR "NewTier" = 2 OR "NewTier" = 1') as cursor:
    for row in cursor:
        
        try:
            #01 - Tier 3, 4 - Two Way Streets - if ped furn + parking < 20, add difference to ped
            if (
                (row[10] is not None) and
                (row[10] == 0) and         #OneWay
                (row[9] in [3,4]) and      #NewTier
                (row[16] is not None) and
                (row[18] is not None) and
                ((row[16] + row[18]) < 20)    #Parking + Ped Furn
                ):
                
                cond = 1
                diff = (20-(row[16]+row[18]))
                row[4] = row[4]+diff
                
            #02 - Tier 2 - Two Way Strets + Protected Bike - if ped furn + parking < 20, add difference to ped   
            elif (
                (row[10] is not None) and
                (row[10] == 0) and           #OneWay
                (row[9] in [2]) and          #NewTier
                (row[16] is not None) and
                (row[18] is not None) and
                ((row[16] + row[18]) < 20) and  #Parking + Ped Furn
                ((row[13] == "Protected Bike Lane") or (row[14] == "Protected Bike Lane") or (row[15] == "Protected Bike Lane"))
                ):
                
                
                cond =2
                diff = (20-(row[16]+row[18]))
                row[4] = row[4]+diff
            
            #03 - Tier 3, 4 - One Way Streets - if ped furn + parking < 10, add difference to ped
            elif(
                (row[10] is not None) and
                (row[10] in [1,2]) and
                (row[9] in [3,4]) and
                (row[16] is not None) and
                (row[18] is not None) and
                ((row[16] + row[18]) <10)
                ):
                
                cond=3
                diff = (10-(row[16]+row[18]))
                row[4] = row[4]+diff
                           
                           
            #04 - Tier 2 - One Way Streets + Protected Bike - if ped furn + parking < 10, add difference to ped   
            elif (
                (row[10] is not None) and
                (row[10] in [1,2]) and
                (row[9] in [2]) and
                (row[16] is not None) and
                (row[18] is not None) and
                ((row[16] + row[18]) < 10) and
                ((row[13] == "Protected Bike Lane") or (row[14] == "Protected Bike Lane") or (row[15] == "Protected Bike Lane"))
                ):
                
                cond=4
                diff = (10-(row[16]+row[18]))
                row[4] = row[4]+diff
                           
            #05 - Tier 1 or (Tier 2 & Not Protected Bike) - Two Way Streets - if ped furn + parking < 16, add difference to ped
            elif (
                (row[10] is not None) and
                (row[10] in [0]) and
                (row[16] is not None) and
                (row[18] is not None) and
                (int((row[16]) + int(row[18])) < 16) and
                ((row[9] == 1) or (row[9] == 2 and not
                        ((row[13] == "Protected Bike Lane") or
                        (row[14] == "Protected Bike Lane") or
                        (row[15] == "Protected Bike Lane"))))
                ):
                
                cond=5
                diff = (16-(row[16]+row[18]))
                print(diff)
                row[4] = row[4]+diff
            
            #06 - Tier 1 or (Tier 2 & Not Protected Bike) - One Way Streets - if ped furn + parking < 8, add difference to ped
            elif (
                (row[10] is not None) and
                (row[10] in [1,2]) and
                (row[16] is not None) and
                (row[18] is not None) and
                ((row[16] + row[18]) <8) and
                ((row[9] == 1) or (row[9] == 2 and not
                        ((row[13] == "Protected Bike Lane") or
                        (row[14] == "Protected Bike Lane") or
                        (row[15] == "Protected Bike Lane"))))
                ):
                
                cond = 6
                diff = (8-(row[16]+row[18]))
                row[4] = row[4]+diff
                           
                           
                           
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0}, Cond:{1}, Parking:{2}, Furniture:{3} - {4} ".format(row[0],cond,row[16], row[18], error))
                
            
            
#proposed ped facilities - use a pass with update cursor to update proposed widths to 12' if they
# have segments listed in Ped improvement datasets, and the existing width is less than 12'
count = 0
err_count = 0

with arcpy.da.UpdateCursor(modal_composite, ped_e_fields,'(NOT "FID_Ped_Improve_temp" IS NULL) OR (NOT "FID_Ped_Add_temp" IS NULL)') as cursor:
    for row in cursor:
        #print("row")
        try:
            #only update proposed width if is less than the current width
            if ((row[4] is not None) and row[4] < 12):
                row[4] = 12
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            

# perform final pass with update cursor to add proposed width to facilites that have no sidewalks
# and are not covered by sidewalk inventory, but are residential (class = 'A41')
count = 0
err_count = 0


#with arcpy.da.UpdateCursor(modal_composite_05a, ped_e_fields,'"NBRightP_1" IS NULL AND "FID_Ped_Improve_temp" IS NULL AND "FID_Ped_Add_temp" IS NULL AND "CLASS" = \'A41\' ') as cursor:
with arcpy.da.UpdateCursor(modal_composite, ped_e_fields,'"NBRightP_1" IS NULL AND "FID_Ped_Improve_temp" IS NULL AND "FID_Ped_Add_temp" IS NULL') as cursor:
    for row in cursor:
        #print("final pass")
        try:
            #only update proposed width if is less than the current width
            if (row[8] == 'A41' and ((row[4] is not None and row[4] < 12) or row[4] is None)):
            #if ((row[8] == 'A41') and (row[4] is None)):
                row[4] = 12   
                #print("pass 3 for A41")
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            

print("Modal Widths - Ped complete.") 

### Calculate Modal Widths for Bus Transit segments

TEXT TO BE REPLACED

- Existing width of 7' is calculated into `modal_width_park_exist` and `modal_width_park_prop` for the following segment types:
    - Roadway Buffer = "Yes" `NBRightP_7 == 1 OR SBLeftPed_6 == 1`
    - Pedestrian Zone Intrusion = "Yes" `NBRightP_4 == 1 or SBLeftPed_4`
    
- If a segment is not listed in the Pedestrian Plan Sidewalk inventory, and it's street type is residential (A41), and mean_row_2020 - modal_width_auto_exist > 22, on street parking is assumed on both sides.




**Assumptions, Notes, and Parameters**
1. Existing facilites from Pedestrian Sidewalk Inventory with data in "Pedestrian Zone Width" field is mapped as follows:
  
|Pedestrian Zone Width| Modal Width Value  |
|:--|:--|
| "Less than 5' " |4'  |
| "5' to 10' " |8'  |
| "Greater than 10' " |12'  |

2. Width values for "NB" and "SB" from Pedestrian Sidewalk Inventory are summed.
3. Street segments listed in "Candidate Walkways", and "Candidate Upgrades" datasets are assumed to be 6' sidewalk on both sides, (12' total)
4. Street segmnents listed in Pedestrian Sidewalk Inventory having no facilites, but are not listed in "Candidate Walkways", or "Candidate Upgrades" and are PLACEHOLDER are assumed to be upgraded to one 6' sidewalk. 6' total.


In [ ]:
#calculate modal width for bus facilities


#expose fields to update cursor
bus_fields = ["SEGMENTID",              #0 - SegmentID
              "NAME",                   #1 - Name
              "modal_width_bus_exist",  #2 - Modal Width - Bus (Existing)
              "modal_width_bus_prop",   #3 - Modal Width - Bus (Proposed)
              "mean_row_2020",          #4 - Mean ROW Analysis
              "CLASS",                  #5 - Street Class (tiger)
              "NewTier",                #6 - Bus Transit - New Tier
              "ONEWAY"]                 #7 - OneWay

bus_count = 0
new_count = 0
err_count = 0

#existing bus facilites by SegmentID (replace with Bus Plan data when available)

IDs_exist = (
    18813,18858,18867,18914,18968,18984,19024,19033,19068,19090,19140,19148,19148,19234,19272,
    19306,19320,19333,19357,19401,19441,19501,19595,19672,19758,19790,21993,22030,22049,22112,
    22163,22169,33146)

where_clause = '"SEGMENTID" IN {}'.format(IDs_exist)

print("Update rows for Bus widths..")
print("Calculating existing...")
#calculate existing Priority Bus facilities
with arcpy.da.UpdateCursor(modal_composite_05a, bus_fields, '"SEGMENTID" IN {}'.format(IDs_exist)) as cursor:   
    for row in cursor:
        #print("row")
        try:
            if row[1] == "HOTEL":
                row[2] = 24
                row[3] = 24
                bus_count += 1
            
            elif row[1] == "KALAKAUA" or row[1] == "KING":
                row[2] = 12
                row[3] = 12
                bus_count += 1
                
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))

print("Existing calc done.")
            
#calculate proposed Tier4 Bus facilities
print("Calculating proposed...")
with arcpy.da.UpdateCursor(modal_composite_05a, bus_fields, '"NewTier" = 4') as cursor:
    for row in cursor:
        try:
            if ((row[7] is not None) and row[7] == 0):
                row[3] = 24
                new_count += 1
            elif((row[7] is not None) and row[7] in [1,2]):
                row[3] = 12
                new_count += 1
                
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            
print("Proposed calc done.") 
            
print("Bus widths - Existing -  calculated for {0} segments".format(bus_count))
print("Bus widths - New -       calculated for {0} segments".format(new_count))
print("Errors (total) for {0} items".format(err_count))
print("Modal Widths - Bus complete.")

### Calculate Unconstrained Modal Width

Use previously calculated modal widths for each modal type to generate a total unconstrained modal width value.


**Assumptions, Notes, and Parameters**
1. Future Tier 4 Bus Priority lanes are ignored in the unconstrained width calcuation based on the assumption that Bus Priority lanes will always be placed in existing auto lanes.
2. Do not sum Ped Width and Bike Width if future Bike Type = Shared Use Path
3. Ignore all bike facility types of shared road way in calculation



In [ ]:
#calcualte unconstrained modal widths

fields = ["SEGMENTID",               #0
          "modal_width_ped_prop",    #1
          "modal_width_bike_prop",   #2
          "modal_width_auto_prop",   #3
          "modal_width_bus_prop",    #4
          "modal_width_park_prop",   #5
          "modal_width_medians",     #6
          "modal_width_max",         #7
          "Fac_Type_BP",             #8
          "Fac_Type_BR",             #9
          "NewTier"]                  #10        

count = 0
err_count = 0

#desc

print("Calculating width..")

with arcpy.da.UpdateCursor(modal_composite, fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            #igonore shared road-way widths from bike width
            if row[8] == "Shared Roadway" or row[9] == "Shared Roadway":     
                row[7] = sum(filter(None,(row[1], row[3], row[4], row[5], row[6])))
                count +=1
                
            #ignore bike shared use path for ped > 7    
            elif (row[8] == "Shared Use Path" or row[9] == "Shared Use Path") and row[1] is not None and row[1] > 7:
                row[7] = sum(filter(None,(row[1], row[3], row[4], row[5], row[6])))
                count +=1
                
            #if shared use path is proposed on segment where future ped width is less than 7, assume 7 for ped width in calc   
            elif (row[8] == "Shared Use Path" or row[9] == "Shared Use Path") and row[1] is not None and row[1] < 7:
                row[7] = sum(filter(None,(row[1], 7,  row[3], row[4], row[5], row[6])))
                count +=1
            
            #summarize all other segments
            else:
                row[7] = sum(filter(None,(row[1], row[2], row[3], row[4], row[5], row[6])))
                count +=1
                
            #row[7] = sum(filter(None,(row[1], row[2], row[3], row[4], row[5], row[6])))
            #row[7] = sum(filter(None,(row[1], row[2], row[3], row[5], row[6])))
            
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            
            
print("Unconstrained widths calculated for {0} items".format(count))
print("Errors (total) for {0} items".format(err_count))
print("Modal Widths - Unconstrained complete.")


In [ ]:
#add additonal fields for modal width/area metrics

#length_ft - to capture line length in ft prior to conversion to WebMercator. Calculated from shape_length
#mean_row_area - estimated area, 2020 row estimate X length
#modal_area_max - estimated unconstrained area, unconstrained width x length
#modal_area_diff - difference between Unconstrained Area and 2020 Estimate. Uncon - 2020. Positive values will represent exceedences, negative will be surplus 


print("adding fields")
arcpy.AddField_management(modal_composite_05a,field_name="length_ft",field_alias = "Length (ft.)",field_type="FLOAT")
arcpy.AddField_management(modal_composite_05a,field_name="mean_row_area",field_alias = "ROW Area Estimate (sq. ft.)",field_type="FLOAT")
arcpy.AddField_management(modal_composite_05a,field_name="modal_area_max",field_alias = "Unconstrained Modal Area (sq. ft.)",field_type="FLOAT")
arcpy.AddField_management(modal_composite_05a,field_name="modal_area_diff",field_alias = "Unconstrained Modal Area Exceedence (sq. ft.)",field_type="FLOAT")
print("done")

In [ ]:
list_fields = arcpy.ListFields(modal_composite_05a)
field_names = [i.name for i in list_fields]

field_check = "modal_area_diff"

if field_check in field_names:
    print("field already exists")
else:
    print("not found")
    

In [ ]:
#calculate modal area fields

fields = ["SEGMENTID",       #0
          "modal_width_max", #1
          "Shape_Length",    #2
          "length_ft",       #3
          "mean_row_area",   #4
          "modal_area_max",  #5
          "modal_area_diff", #6
          "mean_row_2020"]   #7
   

count = 0
err_count = 0

#desc

print("Calculating areas..")

with arcpy.da.UpdateCursor(modal_composite, fields, 'NOT "mean_row_2020" IS NULL') as cursor:
    for row in cursor:
        #print("row")
        try:
            length_ft = row[2]
            mean_row_area = length_ft * row[7]
            modal_area_max = length_ft * row[1]
            modal_area_diff = modal_area_max - mean_row_area
            
            row[3] = length_ft
            row[4] = mean_row_area
            row[5] = modal_area_max
            row[6] = modal_area_diff
            
            count +=1
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            
            
print("Areas calculated for {0} items".format(count))
print("Errors (total) for {0} items".format(err_count))

print("Modal Areas - complete.")

In [ ]:
#reset all (or some) width and area fields to null (use during development)

fields = ["modal_width_ped_exist",
          "modal_width_ped_prop",
          "modal_width_bike_exist",
          "modal_width_bike_prop",
          "modal_width_auto_exist",
          "modal_width_auto_prop",
          "modal_width_bus_exist",
          "modal_width_bus_prop",
          "modal_width_park_exist",
          "modal_width_park_prop",
          "modal_width_medians",
          "modal_width_max",
          "mean_row_area",
          "modal_area_max",
          "modal_area_diff"]

err_count = 0

print("Reseting modal widths and areas to Null...")

with arcpy.da.UpdateCursor(modal_composite, fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            
            
            row[0] = None    # modal_width_ped_exist
            row[1] = None    # modal_width_ped_prop
#             row[2] = None    # modal_width_bike_exist
#             row[3] = None    # modal_width_bike_prop
#             row[4] = None    # modal_width_auto_exist
#             row[5] = None    # modal_width_auto_prop
#             row[6] = None    # modal_width_bus_exist
#             row[7] = None    # modal_width_bus_prop
            row[8] = None    # modal_width_park_exist
            row[9] = None    # modal_width_park_prop
#             row[10] = None   # modal_width_medians
#             row[11] = None   # modal_width_max
#             row[12] = None   # modal_width_park_prop
#             row[13] = None   # modal_width_medians
#             row[14] = None   # modal_width_max




            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print(error)

print("All widths and areas reset to Null.")
            